## Импорт и определение библиотек

In [1]:
!pip install langchain langchain_experimental langchain-openai openai tiktoken huggingface_hub langchain-huggingface -q

In [2]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

## Выбор модели

In [5]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

#print(course_api_key)
# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      #repo_id="IlyaGusev/saiga_llama3_8b",
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = course_api_key
    )"""

## Задаём общие параметры

In [33]:
# Запрашиваем у пользователя ввод ингредиентов
user_input = input("Введите ингредиенты, разделенные запятыми: ")

# Преобразуем введенную строку в список, удаляя лишние пробелы
ingredients = [ingredient.strip() for ingredient in user_input.split(",")]

# Форматируем строку с использованием join для соединения элементов массива
ingredients_str = ", ".join(ingredients)

# Список категорий IBA
iba_categories = [
    "незабываемые", 
    "новая классика", 
    "напитки новой эры", 
    "не вошедшие"
                  ]

# Просим пользователя выбрать категорию или оставить пустым для поиска по всем
#    """Выберите категорию IBA (введите число):
#        1 - Незабываемые
#        2 - Новая классика
#        3 - Напитки новой эры
#        4 - Не вошедшие"""
category_input = input("Выберите категорию IBA (или оставьте пустым для всех категорий): ")

# Приводим выбор к категории
try:
    iba_category = iba_categories[int(category_input) - 1] if category_input.isdigit() and 1 <= int(category_input) <= 4 else None
except ValueError:
    iba_category = None  # Если ничего не выбрано, ищем по всем категориям

# Задаем системный промт
system_prompt = "Ты опытный бармен, специализирующийся на алкогольных коктейлях."

# Задаем промт пользователя с учётом категории
if iba_category:
    user_prompt = (f"Напиши, какие коктейли из категории '{iba_category}' можно изготовить из представленных ниже ингредиентов. Описание не должно содержать обобщающих слов и фраз. "
                   f"Для каждого из представленных тобой коктейлей после перечиления ингредиентов опиши подробный способ приготовления. "
                   f"Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. "
                   f"Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. "
                   f"\nИнгредиенты: {ingredients_str}")
else:
    user_prompt = (f"Напиши, какие коктейли из всех категорий IBA можно изготовить из представленных ниже ингредиентов. Описание не должно содержать обобщающих слов и фраз. "
                   f"Для каждого из представленных тобой коктейлей после перечиления ингредиентов опиши подробный способ приготовления. "
                   f"Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. "
                   f"Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. "
                   f"\nИнгредиенты: {ingredients_str}")


# Выводим сформированный промт
#print(user_prompt)

# Миксуем рецепт

In [8]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
# Получаем ответ от модели
response = llm(messages).content

# Выводим ответ
print("Рецепт коктейлей:\n")
print(response)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

## Если используем модель с Hugging Face

In [35]:
"""# Если выбрали модель с HuggingFace
# Объединяем системный и пользовательский промты в одну строку
full_prompt = f"{system_prompt}\n{user_prompt}"

# Получаем ответ от модели
response = llm(full_prompt)

# Выводим ответ
print(response)"""

/tmp/ipykernel_12156/1038148958.py:6: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(full_prompt)


, гренэджин, сахар, лимон, апельсин, гин, черёмуха, ягоды, лимонная кислота, сироп из апельсина, спирт коньяк, экстракт кофе, сахарный сироп, джин, крем де какао, мята, сладкий шоколад, горчица, лимонный сок, сироп из гренэджина, кусочки льда, айсберг, спирт рисовый, виски, джин, абсент, ликёр амaretto, ликёр десантис, бананы, сливки, бакко и какао-порошок.

1. *Old Fashioned*
Ингредиенты: джин, сахар, дрожжи, гренэджин, лимон, битка льдом.
Для приготовления Old Fashioned нужно: в бокал, заполненный кусками льда, добавить 1 чайную ложку сахара, 2-3 капельки гренэджина и 1 каплю дрожжей. Налить 50 мл джина и несколько капель лимонного сока. Помешиваем все инициативно, чтобы сахар растворился. Затем льдить бокал и декорировать ломтиком апельсина, каплей гренэджина и кокосовым орехом.
Если хочется сделать коктейль *Boulevardier*, нужно заменить джин на виски.

2. *Mint Julep*
Ингредиенты: джин, сахар, листья мяты, лимон, айсберг.
Для приготовления Mint Julep нужно: в бокал заполнить куска

## Меняем стиль ответа

In [37]:
# Функция выбора стиля ответа

def choose_style():
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "Стиль барного меню"  # Стиль по умолчанию
    }
    #print("Доступные стили ответа:")
    for key, value in styles.items():
        print(f"{key}: {value}")
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "style4")

style = choose_style()
prompt = PromptTemplate(input_variables=['output_text', 'style'],
                        template='''Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:''')
style_changer_chain = prompt | llm

styled_response = style_changer_chain.invoke({'output_text': response, 'style': style}).content
print("\nОтвет в выбранном стиле:")
print(styled_response)

1: стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта
2: гопническо-быдляцкий жаргон
3: экспериментальный стиль нарезок Уильяма Берроуза
4: Стиль барного меню

Ответ в выбранном стиле:


1. *The Old One*
Ингредиенты: абсент, ликёр амaretto, ликёр десантис, бакко, какао-порошок, бананы, сливки, спирт коньяк, горчица, лимонный сок.
Чтобы приготовить коктейль *The Old One*, нужно: в бокал, заполненный кусками льда, добавить 1 чайную ложку абсента, 2 капли ликёра амaretto и 3 капли ликёра десантис. Добавить 1 ломтик банана, 2 ложки сливок и 1/2 чайной ложки какао-порошка. Налить 50 мл спирта коньяк и несколько капель горчицы. Помешиваем все инициативно, чтобы абсент не простоложился. Затем льдить бокал и декорировать ломтиком банана, каплей ликёра амaretto и кокосовым орехом.

2. *The Necronomicon*
Ингредиенты: джин, сахар, дрожжи, гренэджин, лимон, сироп из апельсина, сироп из гренэджина, сахарный сироп, кусочки льда, айсберг, спирт рисовый, виски, ликёр амaretto, ликёр де

# Готовим коктейль с помощью RAG

In [38]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
# Загружаем книгу от крутого бармена
loader = PyPDFLoader('../data/Machine_Learning_System_Design_v11.pdf')
book = loader.load()

# Разные способы поиска цен на ингредиенты

## Поиск с помощью агента
1. Инструменты LangChain:
    - Мы можем использовать инструменты, такие как `DuckDuckGoSearchRun` (для поиска через DuckDuckGo), или создать собственные инструменты для взаимодействия с API магазинов (например, Winelab, SimpleWine и других).
2. Агент LangChain:
    - Агент будет обрабатывать запросы, отправлять их в поисковые инструменты и агрегировать ответы.
3. Поиск стоимости товара:
    - Агент использует инструменты, чтобы найти релевантную информацию о стоимости алкогольных ингредиентов.

In [9]:
pip install -U duckduckgo-search -q

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

# Подключение LLM
#llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)

# Создаем поисковый инструмент DuckDuckGo
search = DuckDuckGoSearchRun()

# Добавляем инструмент в список
tools = [
    Tool(
        name="Search ingredient",
        func=search.run,
        description="Используйте этот инструмент для поиска информации об ингредиентах и ценах на самые популярные из них."
    )
]

# Инициализация агента
price_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",  # Используем Zero-shot агент
    verbose=True
)

# Формируем промт для поиска цен на товар
ingredient = "бурбон"
city = "Санкт-Петербург"
prompt = (
    f"Найдите цену на продукт '{ingredient}' в интернет-магазинах {city}. "
    """Если вариантов несколько, укажите 3 лучших результата. 
    Если в ответе название пишется на языке производителя, то оставь как есть. Название валюты и объём должны быть указаны на русском языке. 
    Укажи ответ в формате: название экземпляра, стоимость в среднем ценовом диапазоне, объём, ссылка на магазин, если имеется; если нет, то ничего не писать. 
    Если вариантов ответа несколько, то отобрази их в виде вертикального списка, где каждый новый вариант отображается с новой строки. """
)

# Выполнение агента
result = price_agent.run(prompt)

print(result)




> Entering new AgentExecutor chain...
Я должен использовать инструмент поиска информации об ингредиентах и ценах на самые популярные из них, чтобы найти цену на бурбон в интернет-магазинах Санкт-Петербурга.
Action: Search ingredient
Action Input: 'бурбон'
Observation: Бурбон завдячує своєю появою двом ключовим факторам. Перший з них — знання та таланти шотландських, ірландських та німецьких іммігрантів. Бурбон - это компиляция старых методов в новом сочетании, вот и все. В свое время этот напиток стал хорошей альтернативой дорогостоящему французскому коньяку, по которому так тосковали ... Чем бурбон отличается от других. Бурбон — это разновидность виски. «Виски — от uisge-beatha, «вода жизни» по-кельтски — это зонтичное название кельтских крепких дистиллятов из зерна, — рассказывает Эркин Тузмухамедов ... Бурбон - воплощение традиций американского винокурения. Напиток очень похож на другие виды виски, но имеет некоторые особенности, позволяющие выделить его в отдельную подгруппу креп

## Поиск с помощью `RAG`

Общая стратегия:

1. Использование `WebLoader` для сбора данных:
   - скачиваем страницы для каждой категории алкоголя с сайтов, таких как `Winelab`.
   - сохраняем их текстовое содержимое (например, названия и цены продуктов) в векторное хранилище.
1. Создание векторного хранилища:
    - разбиваем загруженные данные на чанки.
    - индексируем их с помощью векторизатора (например, FAISS или Chroma).
1. Поиск ингредиентов:
    - при запросе пользователя (например, "водка"), извлекаем соответствующую информацию из хранилища.
1. Использование `LangChain`:
    - для загрузки данных используем WebBaseLoader из LangChain.
    - для индексации данных — инструменты LangChain, такие как Chroma или FAISS.
    - для обработки запросов пользователя — цепочки, основанные на LLM (например, `OpenAI API`).

In [175]:
pip install bs4 langchain-openai faiss-cpu -q

Note: you may need to restart the kernel to use updated packages.


In [184]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Установите ваш ключ API
# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# 1. Сбор данных с категорий сайта
def fetch_and_index_alcohol_data():
    categories = {
        "absinthe": "https://www.winelab.ru/catalog/krepkiy-alkogol-absent",
        "vodka": "https://www.winelab.ru/catalog/krepkiy-alkogol-vodka",
        "whiskey": "https://www.winelab.ru/catalog/krepkiy-alkogol-viski",
    }

    documents = []
    for category, url in categories.items():
        try:
            loader = WebBaseLoader(url)
            category_docs = loader.load()
            documents.extend(category_docs)
            print(f"Загружено {len(category_docs)} документов из {url}.")
        except Exception as e:
            print(f"Ошибка при загрузке данных из {url}: {e}")

    print(f"Всего загружено документов: {len(documents)}")
    return documents

# 2. Разбивка текста на чанки
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"Всего создано чанков: {len(chunks)}")
    return chunks


# 3. Индексация данных с использованием FAISS
def create_faiss_index(chunks):
    if not chunks:
        raise ValueError("Нет данных для индексации. Проверьте загрузку и разбивку документов.")
    
    embeddings = OpenAIEmbeddings(openai_api_key=course_api_key)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore


# 4. Создание цепочки RAG для поиска
def create_retrieval_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)
    chain = RetrievalQA(llm=llm, retriever=retriever)
    return chain

# Основная функция
def main():
    print("Сбор данных...")
    documents = fetch_and_index_alcohol_data()
    print("Разделение на чанки...")
    chunks = split_documents(documents)
    print("Создание векторного индекса...")
    vectorstore = create_faiss_index(chunks)
    print("Создание цепочки RAG...")
    rag_chain = create_retrieval_chain(vectorstore)

    # Поиск цены ингредиента
    ingredient_query = "водка"
    print(f"Поиск для: {ingredient_query}")
    result = rag_chain.run(ingredient_query)
    print(f"Результат: {result}")

if __name__ == "__main__":
    main()

Сбор данных...
Загружено 1 документов из https://www.winelab.ru/catalog/krepkiy-alkogol-absent.
Загружено 1 документов из https://www.winelab.ru/catalog/krepkiy-alkogol-vodka.
Загружено 1 документов из https://www.winelab.ru/catalog/krepkiy-alkogol-viski.
Всего загружено документов: 3
Разделение на чанки...
Всего создано чанков: 0
Создание векторного индекса...


ValueError: Нет данных для индексации. Проверьте загрузку и разбивку документов.

## Поиск `в лоб`

In [159]:
import requests
import re

def search_price_with_specific_cookies(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты с индивидуальными куки для каждого магазина.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search/facets?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []

    for store in stores:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
            }
            response = requests.get(store["url"](ingredient), headers=headers, cookies=store["cookies"], timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)

                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")

                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "джин"
print(search_price_with_specific_cookies(ingredient_name))

Данные о наличии в продаже джин не найдены.


In [130]:
# Разбор коктейля и ингредиентов для поиска цен
pattern = r"\*\*(.*?)\*\*:.*?((?:\n\s*-\s*.*?\n)+)"
match = re.search(pattern, response, re.DOTALL)
if match:
    cocktail_name = match.group(1)
    ingredients_section = match.group(2)
    print(f"\nКоктейль: {cocktail_name}")
    print(f"Ингредиенты:")
    print(ingredients_section)
    
    # Поиск цен для каждого ингредиента
    ingredient_lines = ingredients_section.strip().split("\n")
    for line in ingredient_lines:
        ingredient_match = re.search(r"-\s*\d+\s*мл\s*([^\n]+)", line)
        if ingredient_match:
            ingredient_name = ingredient_match.group(1)
            print(f"\nИщем цену для: {ingredient_name}")
            prices = search_price(ingredient_name)
            print(prices)
        else:
            print(f"\nНе удалось извлечь название ингредиента из строки: {line}")
else:
    print("Не удалось найти информацию о коктейле и ингредиентах в ответе модели.")

Не удалось найти информацию о коктейле и ингредиентах в ответе модели.


## Поиск с помощью Selenium

In [ ]:
pip install selenium -q

Note: you may need to restart the kernel to use updated packages.


In [158]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def search_price_with_selenium(ingredient):
    """
    Поиск цен на ингредиенты с использованием Selenium.
    """
    # Укажите путь к вашему WebDriver
    service = Service('/home/elijah/Documents/chromedriver-linux64/chromedriver')  # Укажите корректный путь к ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Запуск в безголовом режиме
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    try:
        # Магазин Winelаб
        driver.get("https://www.winelab.ru")

        # Подтверждение возраста
        age_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "age-confirm"))
        )
        age_button.click()

        # Поиск ингредиента
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))  # Обратите внимание на имя поля
        )

        # Добавление небольшой задержки для обеспечения доступности элемента
        time.sleep(1)

        # Используем JavaScript для установки значения в поле ввода
        driver.execute_script("arguments[0].value = arguments[1];", search_box, ingredient)
        
        # Теперь вызываем событие input для имитации ввода текста
        search_box.send_keys(Keys.RETURN)

        # Найдите кнопку отправки и нажмите ее (если это необходимо)
        # search_button = driver.find_element(By.CLASS_NAME, "js_search_button")
        # search_button.click()

        # Сбор результатов
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card"))
        )
        
        for product in products[:3]:  # Берём максимум 3 результата
            title = product.find_element(By.CLASS_NAME, "product-card__title").text
            price = product.find_element(By.CLASS_NAME, "price__main-value").text
            
            results.append(f"- {title} {price.strip()} рублей")

    except Exception as e:
        results.append(f"Ошибка при обработке ингредиента {ingredient}: {str(e)}")
    finally:
        driver.quit()

    return results if results else f"Данные о наличии в продаже {ingredient} не найдены."

# Пример использования
ingredient_name = "водка"
print(search_price_with_selenium(ingredient_name))

['Ошибка при обработке ингредиента водка: Message: element not interactable\n  (Session info: chrome=131.0.6778.264)\nStacktrace:\n#0 0x5917199fa1fa <unknown>\n#1 0x59171950a66d <unknown>\n#2 0x59171955390c <unknown>\n#3 0x591719551db7 <unknown>\n#4 0x59171957d582 <unknown>\n#5 0x59171954db38 <unknown>\n#6 0x59171957d74e <unknown>\n#7 0x59171959c007 <unknown>\n#8 0x59171957d323 <unknown>\n#9 0x59171954bde0 <unknown>\n#10 0x59171954cdbe <unknown>\n#11 0x5917199c612b <unknown>\n#12 0x5917199ca0c7 <unknown>\n#13 0x5917199b36cc <unknown>\n#14 0x5917199cac47 <unknown>\n#15 0x59171999867f <unknown>\n#16 0x5917199e9288 <unknown>\n#17 0x5917199e9450 <unknown>\n#18 0x5917199f9076 <unknown>\n#19 0x774ab489ca94 <unknown>\n#20 0x774ab4929c3c <unknown>\n']
